In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U trl
!pip install -q -U safetensors
!pip install -q -U datasets


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
!huggingface-cli login --token hf_
!huggingface-cli whoami

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/ubuntu/.cache/huggingface/token
Login successful
taoofstefan


In [3]:
from datasets import load_dataset

dataset = load_dataset('taoofstefan/function_call_weather', split='train')

Generating train split:   0%|          | 0/843 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

In [4]:
def format_instruction(sample):
	return f"""### Instruction:
Use the Input below to infer using the get_current_weather function call correctly. 

### Question:
{sample['Question']}

### Answer:
{sample['Answer']}
"""

In [5]:
from random import randrange

print(format_instruction(dataset[randrange(len(dataset))]))

### Instruction:
Use the Input below to infer using the get_current_weather function call correctly. 

### Question:
Please provide the current weather conditions in Kuala Lumpur.

### Answer:
{"type": "function", "function": {"name": "get_current_weather", "parameters": {"location": "Kuala Lumpur, Malaysia", "format": "celsius"}}}



In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

use_flash_attention = False

# Hugging Face model id
model_id = "meta-llama/Llama-2-7b-hf"


# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    use_cache=False,
    device_map="auto",
)
model.config.pretraining_tp = 1


tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

The `SFTTrainer`  supports a native integration with `peft`, which makes it super easy to efficiently instruction tune LLMs. We only need to create our `LoRAConfig` and provide it to the trainer.

In [7]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM", 
)

# prepare model for training
model = prepare_model_for_kbit_training(model)

Before we can start our training we need to define the hyperparameters (`TrainingArguments`) we want to use.

In [8]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="llama-7",
    num_train_epochs=50,
    per_device_train_batch_size=6 if use_flash_attention else 4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    fp16=False,
    tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    disable_tqdm=False,  
)

# Upcast layer for flash attnetion
if use_flash_attention:
    from utils.llama_patch import upcast_layer_for_flash_attention
    torch_dtype = torch.bfloat16 if args.bf16 else torch.float16 if args.fp16 else torch.float32
    model = upcast_layer_for_flash_attention(model, torch_dtype)

model = get_peft_model(model, peft_config)


We now have every building block we need to create our `SFTTrainer` to start then training our model.

In [9]:
from trl import SFTTrainer

max_seq_length = 2048 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=format_instruction, 
    args=args,
)

/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, packing. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/transformers/training_args.py:1960: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:181: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed

Generating train split: 0 examples [00:00, ? examples/s]

Start training our model by calling the `train()` method on our `Trainer` instance.

In [10]:
# train
trainer.train()

# save model
trainer.save_model()

/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,0.736400
20,0.430100
30,0.296400
40,0.245000
50,0.210600
60,0.184000
70,0.163300
80,0.149400
90,0.138200
100,0.127200


/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/ubuntu/.pyenv/versions/3

## 4. Test Model and run Inference

After the training is done we want to run and test our model. We will use `peft` and `transformers` to load our LoRA adapter into our model.

In [39]:
if use_flash_attention:
    # unpatch flash attention
    from utils.llama_patch import unplace_flash_attn_with_attn
    unplace_flash_attn_with_attn()
    
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer


args.output_dir = "llama-7"

# load base LLM model and tokenizer
model = AutoPeftModelForCausalLM.from_pretrained(
    args.output_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
) 
tokenizer = AutoTokenizer.from_pretrained(args.output_dir)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Let’s load the dataset again with a random sample to try to generate an instruction.

In [40]:
from datasets import load_dataset 
from random import randrange


# Load dataset from the hub and get a sample
dataset = load_dataset('taoofstefan/function_call_weather', split='train')
sample = dataset[randrange(len(dataset))]

prompt = f"""### Instruction:
Use the Input below to infer using the get_current_weather function call correctly. 

### Question:
{sample['Question']}

### Answer:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.9)

print(f"Prompt:\n{sample['Question']}\n")
print(f"Generated instruction:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"Ground truth:\n{sample['Answer']}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Prompt:
What's the temperature in Amsterdam at the moment?

Generated instruction:
{"type": "function", "function": {"name": "get_current_weather", "parameters": {"location": "Amsterdam, Netherlands", "format": "celsius"}}}

Ground truth:
{"type": "function", "function": {"name": "get_current_weather", "parameters": {"location": "Amsterdam, Netherlands", "format": "celsius"}}}


Nice! our model works! If want to accelerate our model we can deploy it with [Text Generation Inference](https://github.com/huggingface/text-generation-inference). Therefore we would need to merge our adapter weights into the base model.

In [29]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    args.output_dir,
    low_cpu_mem_usage=True,
) 

# Merge LoRA and base model
merged_model = model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

('merged_model/tokenizer_config.json',
 'merged_model/special_tokens_map.json',
 'merged_model/tokenizer.json')

In [34]:
project = "MT-FT-llama2-S"
base_model_name = "llama2-7B"
version = "v0.3"
repo_id = base_model_name + "-" + project + "-" + version

print(run_name)

llama2-7B-MT-FT-llama2-S-v0.3


In [35]:
from huggingface_hub import notebook_login

!huggingface-cli login --token hf_

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/ubuntu/.cache/huggingface/token
Login successful


In [36]:
# push merged model to the hub
merged_model.push_to_hub(repo_id, token=True, max_shard_size="5GB", safe_serialization=True)
tokenizer.push_to_hub(repo_id, token=True)

model-00006-of-00006.safetensors:   0%|          | 0.00/2.68G [00:00<?, ?B/s]

model-00004-of-00006.safetensors:   0%|          | 0.00/4.86G [00:00<?, ?B/s]

model-00003-of-00006.safetensors:   0%|          | 0.00/4.86G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.86G [00:00<?, ?B/s]

Upload 6 LFS files:   0%|          | 0/6 [00:00<?, ?it/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.84G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/4.86G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/taoofstefan/llama2-7B-MT-FT-llama2-S-v0.3/commit/c4565d916f0b1ef2ba8bf21157869787baecdc66', commit_message='Upload tokenizer', commit_description='', oid='c4565d916f0b1ef2ba8bf21157869787baecdc66', pr_url=None, pr_revision=None, pr_num=None)

In [41]:
from datasets import load_dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load dataset from the hub and get a sample
dataset = load_dataset('taoofstefan/function_call_weather', split='test')

# Convert dataset to pandas DataFrame
df = pd.DataFrame(dataset)

# Initialize the Answer_Actual column
df['Answer_Actual'] = ""

# Loop through each row in the DataFrame
for index, row in df.iterrows():
    prompt = f"""### Instruction:
Use the Input below to infer using the get_current_weather function call correctly. 

### Question:
{row['Question']}

### Answer:
"""
    
    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
    # with torch.inference_mode():
    outputs = model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9, temperature=0.9)
    
    generated_answer = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]
    df.at[index, 'Answer_Actual'] = generated_answer


{"type": "function", "function": {"name": "get_current_weather", "parameters": {"location": "Lagos, Nigeria", "format": "celsius"}}}

{"type": "function", "function": "get_current_weather", "parameters": {"location": "Sao Paulo, Brazil", "format": "celsius"}}}

{"type": "function", "function": {"name": "get_current_weather", "parameters": {"location": "Kuala Lumpur, Malaysia", "templates": "Weather processes are active in the area.  "}}}

You don't need to worry about the heat in Nairobi today. The weather is mild and pleasant.

{"type": "function", "function": {"name": "get_current_weather", "parameters": {"location": "Lagos, Nigeria", "format": "fahrenheit"}}}

{"type": "function", "function": {"name": "get_current_weather", "parameters": {"location": "Melbourne, Australia", "format": "celsius"}}}

You can learn about any weather hazards by visiting the official weather forecasting site for Phoenix.

{"type": "function", "function": {"name": "get_current_weather", "parameters": {"loc

ModuleNotFoundError: No module named 'ace_tools'

In [57]:
df_change = df.copy(deep=True)
df_change['Answer_Actual'] = df_change['Answer_Actual'].str.replace('"', "'")
df_change.head()

,Question,Answer,Answer_Actual,Normalized_Answer_Actual,Precision
0,What will be the peak temperature in Lagos today?,None,None,"{""function"": {""name"": ""get_current_weather"", ""...",False
1,How does the weather today in Sao Paulo compar...,None,None,"{'type': 'function', 'function': 'get_current_...",False
2,how warm is it in Kuala Lumpur right now?,None,None,"{""function"": {""name"": ""get_current_weather"", ""...",False
3,What is the UV index in Nairobi today?,None,None,You don't need to worry about the heat in Nair...,False
4,Is Lagos experiencing any extreme weather cond...,None,None,"{""function"": {""name"": ""get_current_weather"", ""...",False


In [51]:
df_change.to_csv("result_llama2-7b-mt-ft-llama2-s_v0.3-q4_k_m.csv", sep=";", index=False)